In [ ]:
# | default_exp nets/swinv2_3d

# Imports

In [ ]:
# | export

import numpy as np
import torch
from huggingface_hub import PyTorchModelHubMixin  # TODO
from torch import nn

from vision_architectures.blocks.transformer import Attention3DWithMLP, Attention3DWithMLPConfig
from vision_architectures.docstrings import populate_docstring
from vision_architectures.layers.embeddings import (
    AbsolutePositionEmbeddings3D,
    PatchEmbeddings3D,
    RelativePositionEmbeddings3DConfig,
    RelativePositionEmbeddings3DMetaNetwork,
)
from vision_architectures.nets.swin_3d import (
    Swin3DBlock,
    Swin3DBlockConfig,
    Swin3DEncoderDecoderBase,
    Swin3DEncoderDecoderConfig,
    Swin3DEncoderWithPatchEmbeddings,
    Swin3DEncoderWithPatchEmbeddingsConfig,
    Swin3DLayer,
    Swin3DPatchMerging,
    Swin3DPatchMergingConfig,
    Swin3DPatchSplitting,
    Swin3DPatchSplittingConfig,
    Swin3DStage,
    Swin3DStageConfig,
)
from vision_architectures.utils.custom_base_model import Field

# Config

In [ ]:
# | export


class SwinV23DPatchMergingConfig(Swin3DPatchMergingConfig):
    pass


class SwinV23DPatchSplittingConfig(Swin3DPatchSplittingConfig):
    pass


class SwinV23DBlockConfig(Swin3DBlockConfig):
    patch_merging: SwinV23DPatchMergingConfig | None = Field(
        None, description="Patch merging config if desired. Patch merging is applied before attention."
    )
    patch_splitting: SwinV23DPatchSplittingConfig | None = Field(
        None, description="Patch splitting config if desired. Patch splitting is applied after attention."
    )


class SwinV23DStageConfig(SwinV23DBlockConfig, Swin3DStageConfig):
    pass


class SwinV23DEncoderDecoderConfig(Swin3DEncoderDecoderConfig):
    stages: list[SwinV23DStageConfig]


class SwinV23DEncoderWithPatchEmbeddingsConfig(SwinV23DEncoderDecoderConfig, Swin3DEncoderWithPatchEmbeddingsConfig):
    pass

In [ ]:
test_config = SwinV23DEncoderWithPatchEmbeddingsConfig.model_validate(
    {
        "patch_size": (1, 8, 8),
        "in_channels": 1,
        "use_absolute_position_embeddings": True,
        "learnable_absolute_position_embeddings": False,
        "stages": [
            {
                "dim": 36,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "in_dim": 36,
                    "out_dim": 72,
                },
                "dim": 72,
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "in_dim": 72,
                    "out_dim": 144,
                },
                "dim": 144,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
                "patch_splitting": {
                    "final_window_size": (2, 2, 2),
                    "in_dim": 144,
                    "out_dim": 72,
                },
            },
        ],
    }
)

test_config


SwinV23DEncoderWithPatchEmbeddingsConfig(
    stages=[
        SwinV23DStageConfig(
            dim=36,
            num_heads=4,
            ratio_q_to_kv_heads=1,
            logit_scale_learnable=False,
            attn_drop_prob=0.0,
            proj_drop_prob=0.0,
            max_attention_batch_size=-1,
            mlp_ratio=4,
            activation='gelu',
            mlp_drop_prob=0.0,
            norm_location='post',
            layer_norm_eps=1e-06,
            window_size=(4, 4, 4),
            use_relative_position_bias=False,
            patch_merging=None,
            patch_splitting=None,
            in_dim=None,
            out_dim=None,
            depth=1
        ),
        SwinV23DStageConfig(
            dim=72,
            num_heads=4,
            ratio_q_to_kv_heads=1,
            logit_scale_learnable=False,
            attn_drop_prob=0.0,
            proj_drop_prob=0.0,
            max_attention_batch_size=-1,
            mlp_ratio=4,
            activation='g

# Architecture

### Basic Layers

In [ ]:
# | export


class SwinV23DLayerLogitScale(nn.Module):
    def __init__(self, num_heads):
        super().__init__()

        self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_heads, 1, 1))), requires_grad=True)

    def forward(self):
        logit_scale = torch.clamp(self.logit_scale, max=np.log(1.0 / 0.01)).exp()
        return logit_scale

In [ ]:
# | export


@populate_docstring
class SwinV23DLayer(Swin3DLayer):
    """SwinV2 3D Layer applying windowed attention with optional relative position embeddings.
    {CLASS_DESCRIPTION_3D_DOC}"""

    @populate_docstring
    def __init__(
        self,
        config: RelativePositionEmbeddings3DConfig | Attention3DWithMLPConfig = {},
        checkpointing_level: int = 0,
        **kwargs
    ):
        """Initializes the SwinV23DLayer.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__(config, checkpointing_level, **kwargs)

        # Update relative position bias to use the meta network
        relative_position_bias = None
        if self._use_relative_position_bias:
            relative_position_bias = RelativePositionEmbeddings3DMetaNetwork(
                self.embeddings_config, checkpointing_level=checkpointing_level
            )

        # Use SwinV2 logit scale
        logit_scale = SwinV23DLayerLogitScale(self.transformer_config.num_heads)

        # Re-initialize the transformer with the new relative position bias and logit scale
        self.transformer = Attention3DWithMLP(
            self.transformer_config,
            relative_position_bias=relative_position_bias,
            logit_scale=logit_scale,
            checkpointing_level=checkpointing_level,
        )

In [ ]:
test = SwinV23DLayer(
    dim=64,
    num_heads=4,
    mlp_ratio=4,
    layer_norm_eps=1e-6,
    window_size=(2, 2, 2),
    use_relative_position_bias=True,
)
display(test)
display(test(torch.randn(2, 64, 4, 4, 4)).shape)


SwinV23DLayer(
  (transformer): Attention3DWithMLP(
    (attn): Attention3D(
      (W_q): Linear(in_features=64, out_features=64, bias=True)
      (W_k): Linear(in_features=64, out_features=64, bias=True)
      (W_v): Linear(in_features=64, out_features=64, bias=True)
      (proj): Linear(in_features=64, out_features=64, bias=True)
      (proj_drop): Dropout(p=0.0, inplace=False)
      (logit_scale): SwinV23DLayerLogitScale()
      (relative_position_bias): RelativePositionEmbeddings3DMetaNetwork(
        (cpb_mlp): Sequential(
          (0): Linear(in_features=3, out_features=512, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=512, out_features=4, bias=False)
        )
        (checkpointing_level1): ActivationCheckpointing(enabled=False)
      )
      (checkpointing_level1): ActivationCheckpointing(enabled=False)
      (checkpointing_level2): ActivationCheckpointing(enabled=False)
    )
    (layernorm1): LayerNorm((64,), eps=1e-06, elementwise_affine=

torch.Size([2, 64, 4, 4, 4])

### Stage layers

In [ ]:
# | export


@populate_docstring
class SwinV23DBlock(Swin3DBlock):
    """SwinV2 3D Block consisting of two SwinV23DLayers: one with regular windows and one with shifted windows.
    {CLASS_DESCRIPTION_3D_DOC}"""

    @populate_docstring
    def __init__(self, config: SwinV23DBlockConfig = {}, checkpointing_level: int = 0, **kwargs):
        """Initializes the SwinV23DBlock.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__(config, checkpointing_level, **kwargs)

        self.config = SwinV23DBlockConfig.model_validate(config | kwargs)

        self.w_layer = SwinV23DLayer(self.config.model_dump(), checkpointing_level=checkpointing_level)
        self.sw_layer = SwinV23DLayer(self.config.model_dump(), checkpointing_level=checkpointing_level)

In [ ]:
test_stage_config = SwinV23DBlockConfig.model_validate(
    {
        "dim": 64,
        "depth": 4,
        "num_heads": 4,
        "mlp_ratio": 4,
        "layer_norm_eps": 1e-6,
        "window_size": (4, 4, 4),
        "use_relative_position_bias": True,
    }
)

test = SwinV23DBlock(test_stage_config)
display(test)
o = test(torch.randn(2, 64, 4, 4, 4), return_intermediates=True)
display((o[0].shape, (o[1][0].shape, o[1][1].shape)))


SwinV23DBlock(
  (w_layer): SwinV23DLayer(
    (transformer): Attention3DWithMLP(
      (attn): Attention3D(
        (W_q): Linear(in_features=64, out_features=64, bias=True)
        (W_k): Linear(in_features=64, out_features=64, bias=True)
        (W_v): Linear(in_features=64, out_features=64, bias=True)
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
        (logit_scale): SwinV23DLayerLogitScale()
        (relative_position_bias): RelativePositionEmbeddings3DMetaNetwork(
          (cpb_mlp): Sequential(
            (0): Linear(in_features=3, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=4, bias=False)
          )
          (checkpointing_level1): ActivationCheckpointing(enabled=False)
        )
        (checkpointing_level1): ActivationCheckpointing(enabled=False)
        (checkpointing_level2): ActivationCheckpointing(enabled=False)
      )

(torch.Size([2, 64, 4, 4, 4]), (torch.Size([2, 4, 4, 4, 64]), torch.Size([2, 4, 4, 4, 64])))

In [ ]:
# | export


@populate_docstring
class SwinV23DPatchMerging(Swin3DPatchMerging):
    """Patch merging layer for SwinV23D. {CLASS_DESCRIPTION_3D_DOC}"""

    @populate_docstring
    def __init__(self, config: SwinV23DPatchMergingConfig = {}, checkpointing_level: int = 0, **kwargs):
        """Initialize the SwinV23DPatchMerging layer.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            **kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__(config, checkpointing_level, **kwargs)

        self.config = SwinV23DPatchMergingConfig.model_validate(config | kwargs)

        in_dim = self.config.in_dim * np.prod(self.config.merge_window_size)
        self.layer_norm = nn.LayerNorm(in_dim)
        self.proj = nn.Linear(in_dim, self.config.out_dim)

In [ ]:
test_stage_config = SwinV23DPatchMergingConfig.model_validate(
    {
        "merge_window_size": (2, 2, 2),
        "in_dim": 64,
        "out_dim": 108,
    }
)

test = SwinV23DPatchMerging(test_stage_config)
display(test)
display(test(torch.randn(2, 64, 4, 4, 4)).shape)


SwinV23DPatchMerging(
  (layer_norm): LayerNorm((np.int64(512),), eps=1e-05, elementwise_affine=True)
  (proj): Linear(in_features=512, out_features=108, bias=True)
  (checkpointing_level1): ActivationCheckpointing(enabled=False)
)

torch.Size([2, 108, 2, 2, 2])

In [ ]:
# | export


@populate_docstring
class SwinV23DPatchSplitting(Swin3DPatchSplitting):
    """Patch splitting layer for SwinV23D. {CLASS_DESCRIPTION_3D_DOC}

    This is a self-implemented class and is not part of the paper."""

    @populate_docstring
    def __init__(self, config: SwinV23DPatchSplittingConfig, checkpointing_level: int = 0, **kwargs):
        """Initialize the SwinV23DPatchSplitting layer.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            **kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__(config, checkpointing_level, **kwargs)

        self.config = SwinV23DPatchSplittingConfig.model_validate(config | kwargs)

        self.layer_norm = nn.LayerNorm(self.config.in_dim)
        self.proj = nn.Linear(self.config.in_dim, self.config.out_dim * np.prod(self.config.final_window_size))

In [ ]:
test_stage_config = SwinV23DPatchSplittingConfig.model_validate(
    {
        "final_window_size": (2, 2, 2),
        "in_dim": 72,
        "out_dim": 64,
    }
)

test = SwinV23DPatchSplitting(test_stage_config)
display(test)
display(test(torch.randn(2, 72, 4, 4, 4)).shape)


SwinV23DPatchSplitting(
  (layer_norm): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
  (proj): Linear(in_features=72, out_features=512, bias=True)
  (checkpointing_level1): ActivationCheckpointing(enabled=False)
)

torch.Size([2, 64, 8, 8, 8])

In [ ]:
# | export


@populate_docstring
class SwinV23DStage(Swin3DStage):
    """SwinV23D stage for SwinV23D. {CLASS_DESCRIPTION_3D_DOC}"""

    @populate_docstring
    def __init__(self, config: SwinV23DStageConfig = {}, checkpointing_level: int = 0, **kwargs):
        """Initialize the SwinV23DStage.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            **kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__(config, checkpointing_level, **kwargs)

        self.config = SwinV23DStageConfig.model_validate(config | kwargs)

        self.patch_merging = None
        if self.config.patch_merging is not None:
            self.patch_merging = SwinV23DPatchMerging(self.config.patch_merging)

        self.blocks = nn.ModuleList(
            [SwinV23DBlock(self.config) for _ in range(self.config.depth)],
        )

        self.patch_splitting = None
        if self.config.patch_splitting is not None:
            # This has been implemented to create a Swin-based decoder
            self.patch_splitting = SwinV23DPatchSplitting(self.config.patch_splitting)

In [ ]:
test_stage_config = SwinV23DStageConfig.model_validate(
    {
        "patch_merging": {
            "merge_window_size": (2, 2, 2),
            "in_dim": 48,
            "out_dim": 108,
        },
        "dim": 108,
        "depth": 2,
        "num_heads": 4,
        "mlp_ratio": 4,
        "layer_norm_eps": 1e-6,
        "window_size": (4, 4, 4),
        "use_relative_position_bias": True,
    }
)
display(test_stage_config)

test = SwinV23DStage(test_stage_config)
display(test)
o = test(torch.randn(2, 48, 8, 8, 8), return_intermediates=True)
display((o[0].shape, [x.shape for x in o[1]]))


SwinV23DStageConfig(
    dim=108,
    num_heads=4,
    ratio_q_to_kv_heads=1,
    logit_scale_learnable=False,
    attn_drop_prob=0.0,
    proj_drop_prob=0.0,
    max_attention_batch_size=-1,
    mlp_ratio=4,
    activation='gelu',
    mlp_drop_prob=0.0,
    norm_location='post',
    layer_norm_eps=1e-06,
    window_size=(4, 4, 4),
    use_relative_position_bias=True,
    patch_merging=SwinV23DPatchMergingConfig(
        in_dim=48,
        out_dim=108,
        merge_window_size=(2, 2, 2),
        out_dim_ratio=2.25
    ),
    patch_splitting=None,
    in_dim=48,
    out_dim=None,
    depth=2
)


SwinV23DStage(
  (patch_merging): SwinV23DPatchMerging(
    (layer_norm): LayerNorm((np.int64(384),), eps=1e-05, elementwise_affine=True)
    (proj): Linear(in_features=384, out_features=108, bias=True)
    (checkpointing_level1): ActivationCheckpointing(enabled=False)
  )
  (blocks): ModuleList(
    (0-1): 2 x SwinV23DBlock(
      (w_layer): SwinV23DLayer(
        (transformer): Attention3DWithMLP(
          (attn): Attention3D(
            (W_q): Linear(in_features=108, out_features=108, bias=True)
            (W_k): Linear(in_features=108, out_features=108, bias=True)
            (W_v): Linear(in_features=108, out_features=108, bias=True)
            (proj): Linear(in_features=108, out_features=108, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (logit_scale): SwinV23DLayerLogitScale()
            (relative_position_bias): RelativePositionEmbeddings3DMetaNetwork(
              (cpb_mlp): Sequential(
                (0): Linear(in_features=3, out_featu


(
    torch.Size([2, 108, 4, 4, 4]),
    [
        torch.Size([2, 4, 4, 4, 108]),
        torch.Size([2, 4, 4, 4, 108]),
        torch.Size([2, 4, 4, 4, 108]),
        torch.Size([2, 4, 4, 4, 108])
    ]
)

In [ ]:
test_stage_config = SwinV23DStageConfig.model_validate(
    {
        "patch_merging": None,
        "dim": 48,
        "depth": 2,
        "num_heads": 4,
        "mlp_ratio": 4,
        "layer_norm_eps": 1e-6,
        "window_size": (4, 4, 4),
        "use_relative_position_bias": True,
        "patch_splitting": {
            "final_window_size": (2, 2, 2),
            "in_dim": 48,
            "out_dim": 18,
        },
    }
)

test = SwinV23DStage(test_stage_config)
display(test)
o = test(torch.randn(2, 48, 8, 8, 8), return_intermediates=True)
display((o[0].shape, [x.shape for x in o[1]]))
o = test(torch.randn(2, 48, 8, 8, 8), return_intermediates=True)
display((o[0].shape, [x.shape for x in o[1]]))


SwinV23DStage(
  (blocks): ModuleList(
    (0-1): 2 x SwinV23DBlock(
      (w_layer): SwinV23DLayer(
        (transformer): Attention3DWithMLP(
          (attn): Attention3D(
            (W_q): Linear(in_features=48, out_features=48, bias=True)
            (W_k): Linear(in_features=48, out_features=48, bias=True)
            (W_v): Linear(in_features=48, out_features=48, bias=True)
            (proj): Linear(in_features=48, out_features=48, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (logit_scale): SwinV23DLayerLogitScale()
            (relative_position_bias): RelativePositionEmbeddings3DMetaNetwork(
              (cpb_mlp): Sequential(
                (0): Linear(in_features=3, out_features=512, bias=True)
                (1): ReLU(inplace=True)
                (2): Linear(in_features=512, out_features=4, bias=False)
              )
              (checkpointing_level1): ActivationCheckpointing(enabled=False)
            )
            (checkpointing_


(
    torch.Size([2, 18, 16, 16, 16]),
    [
        torch.Size([2, 8, 8, 8, 48]),
        torch.Size([2, 8, 8, 8, 48]),
        torch.Size([2, 8, 8, 8, 48]),
        torch.Size([2, 8, 8, 8, 48])
    ]
)


(
    torch.Size([2, 18, 16, 16, 16]),
    [
        torch.Size([2, 8, 8, 8, 48]),
        torch.Size([2, 8, 8, 8, 48]),
        torch.Size([2, 8, 8, 8, 48]),
        torch.Size([2, 8, 8, 8, 48])
    ]
)

### Encoder/Decoder Base

In [ ]:
# | export


class SwinV23DEncoderDecoderBase(Swin3DEncoderDecoderBase, PyTorchModelHubMixin):
    @populate_docstring
    def __init__(self, config: SwinV23DEncoderDecoderConfig = {}, checkpointing_level: int = 0, **kwargs):
        """Initializes the SwinV23DEncoder/SwinV23DDecoder.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__(config, checkpointing_level, **kwargs)

        self.config = SwinV23DEncoderDecoderConfig.model_validate(config | kwargs)

        self.stages = nn.ModuleList(
            [SwinV23DStage(stage_config, checkpointing_level) for stage_config in self.config.stages]
        )

### Encoder

In [ ]:
# | export


@populate_docstring
class SwinV23DEncoder(SwinV23DEncoderDecoderBase):
    """3D Swin Transformer encoder. Assumes input has already been patchified/tokenized. {CLASS_DESCRIPTION_3D_DOC}"""

    def __init__(self, config: SwinV23DEncoderDecoderConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__(config, checkpointing_level, **kwargs)

        for stage_config in self.config.stages:
            if stage_config.patch_splitting is not None:
                assert (
                    stage_config.patch_merging is not None
                ), "SwinV23DEncoder is not for decoding (mid blocks are ok)."

In [ ]:
test_config = SwinV23DEncoderDecoderConfig.model_validate(
    {
        "patch_size": (2, 2, 2),
        "in_channels": 32,
        "stages": [
            {
                "dim": 32,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "in_dim": 32,
                    "out_dim": 64,
                },
                "dim": 64,
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "in_dim": 64,
                    "out_dim": 128,
                },
                "dim": 128,
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
                "patch_splitting": {
                    "final_window_size": (2, 2, 2),
                    "in_dim": 128,
                    "out_dim": 100,
                },
            },
        ],
    }
)

test = SwinV23DEncoder(test_config)
display(test)
o = test(torch.randn(2, 32, 16, 16, 16), return_intermediates=True)
display((o[0].shape, [x.shape for x in o[1]], [x.shape for x in o[2]]))


SwinV23DEncoder(
  (stages): ModuleList(
    (0): SwinV23DStage(
      (blocks): ModuleList(
        (0): SwinV23DBlock(
          (w_layer): SwinV23DLayer(
            (transformer): Attention3DWithMLP(
              (attn): Attention3D(
                (W_q): Linear(in_features=32, out_features=32, bias=True)
                (W_k): Linear(in_features=32, out_features=32, bias=True)
                (W_v): Linear(in_features=32, out_features=32, bias=True)
                (proj): Linear(in_features=32, out_features=32, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (logit_scale): SwinV23DLayerLogitScale()
                (checkpointing_level1): ActivationCheckpointing(enabled=False)
                (checkpointing_level2): ActivationCheckpointing(enabled=False)
              )
              (layernorm1): LayerNorm((32,), eps=1e-06, elementwise_affine=True)
              (mlp): Attention3DMLP(
                (dense1): Linear(in_features=32, out_fe


(
    torch.Size([2, 100, 8, 8, 8]),
    [torch.Size([2, 16, 16, 16, 32]), torch.Size([2, 8, 8, 8, 64]), torch.Size([2, 8, 8, 8, 100])],
    [
        torch.Size([2, 16, 16, 16, 32]),
        torch.Size([2, 16, 16, 16, 32]),
        torch.Size([2, 8, 8, 8, 64]),
        torch.Size([2, 8, 8, 8, 64]),
        torch.Size([2, 8, 8, 8, 64]),
        torch.Size([2, 8, 8, 8, 64]),
        torch.Size([2, 8, 8, 8, 64]),
        torch.Size([2, 8, 8, 8, 64]),
        torch.Size([2, 4, 4, 4, 128]),
        torch.Size([2, 4, 4, 4, 128]),
        torch.Size([2, 4, 4, 4, 128]),
        torch.Size([2, 4, 4, 4, 128]),
        torch.Size([2, 4, 4, 4, 128]),
        torch.Size([2, 4, 4, 4, 128])
    ]
)

### Decoder

In [ ]:
# | export


@populate_docstring
class SwinV23DDecoder(SwinV23DEncoderDecoderBase):
    """3D Swin Transformer decoder. Assumes input has already been patchified/tokenized. {CLASS_DESCRIPTION_3D_DOC}"""

    def __init__(self, config: SwinV23DEncoderDecoderConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__(config, checkpointing_level, **kwargs)

        for stage_config in config.stages:
            if stage_config.patch_merging is not None:
                assert (
                    stage_config.patch_splitting is not None
                ), "SwinV23DDecoder is not for encoding (mid blocks are ok)."

In [ ]:
test_config = SwinV23DEncoderDecoderConfig.model_validate(
    {
        "stages": [
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "in_dim": 96,
                    "out_dim": 288,
                },
                "dim": 288,
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
                "patch_splitting": {
                    "final_window_size": (2, 2, 2),
                    "in_dim": 288,
                    "out_dim": 96,
                },
            },
            {
                "dim": 96,
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
                "patch_splitting": {
                    "final_window_size": (2, 2, 2),
                    "in_dim": 96,
                    "out_dim": 32,
                },
            },
            {
                "dim": 32,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
            },
        ],
    }
)

test = SwinV23DDecoder(test_config)
display(test)
o = test(torch.randn(2, 96, 16, 16, 16), return_intermediates=True)
display((o[0].shape, [x.shape for x in o[1]], [x.shape for x in o[2]]))


SwinV23DDecoder(
  (stages): ModuleList(
    (0): SwinV23DStage(
      (patch_merging): SwinV23DPatchMerging(
        (layer_norm): LayerNorm((np.int64(768),), eps=1e-05, elementwise_affine=True)
        (proj): Linear(in_features=768, out_features=288, bias=True)
        (checkpointing_level1): ActivationCheckpointing(enabled=False)
      )
      (blocks): ModuleList(
        (0-2): 3 x SwinV23DBlock(
          (w_layer): SwinV23DLayer(
            (transformer): Attention3DWithMLP(
              (attn): Attention3D(
                (W_q): Linear(in_features=288, out_features=288, bias=True)
                (W_k): Linear(in_features=288, out_features=288, bias=True)
                (W_v): Linear(in_features=288, out_features=288, bias=True)
                (proj): Linear(in_features=288, out_features=288, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (logit_scale): SwinV23DLayerLogitScale()
                (relative_position_bias): RelativePosi


(
    torch.Size([2, 32, 32, 32, 32]),
    [
        torch.Size([2, 16, 16, 16, 96]),
        torch.Size([2, 32, 32, 32, 32]),
        torch.Size([2, 32, 32, 32, 32])
    ],
    [
        torch.Size([2, 8, 8, 8, 288]),
        torch.Size([2, 8, 8, 8, 288]),
        torch.Size([2, 8, 8, 8, 288]),
        torch.Size([2, 8, 8, 8, 288]),
        torch.Size([2, 8, 8, 8, 288]),
        torch.Size([2, 8, 8, 8, 288]),
        torch.Size([2, 16, 16, 16, 96]),
        torch.Size([2, 16, 16, 16, 96]),
        torch.Size([2, 16, 16, 16, 96]),
        torch.Size([2, 16, 16, 16, 96]),
        torch.Size([2, 16, 16, 16, 96]),
        torch.Size([2, 16, 16, 16, 96]),
        torch.Size([2, 32, 32, 32, 32]),
        torch.Size([2, 32, 32, 32, 32])
    ]
)

# Models

In [ ]:
# | export


@populate_docstring
class SwinV23DEncoderWithPatchEmbeddings(Swin3DEncoderWithPatchEmbeddings, PyTorchModelHubMixin):
    """3D SwinV2 transformer with 3D patch embeddings. {CLASS_DESCRIPTION_3D_DOC}"""

    @populate_docstring
    def __init__(self, config: SwinV23DEncoderWithPatchEmbeddingsConfig = {}, checkpointing_level: int = 0, **kwargs):
        """Initializes the SwinV23DEncoderWithPatchEmbeddings.

        Args:
            config: {CONFIG_INSTANCE_DOC}
            checkpointing_level: {CHECKPOINTING_LEVEL_DOC}
            kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__(config, checkpointing_level, **kwargs)

        self.config = SwinV23DEncoderWithPatchEmbeddingsConfig.model_validate(config | kwargs)

        self.patchify = PatchEmbeddings3D(
            patch_size=self.config.patch_size,
            in_channels=self.config.in_channels,
            dim=self.config.stages[0].get_in_dim(),
            checkpointing_level=checkpointing_level,
        )
        self.absolute_position_embeddings = AbsolutePositionEmbeddings3D(
            dim=self.config.stages[0].get_in_dim(), learnable=False
        )
        self.encoder = SwinV23DEncoder(self.config, checkpointing_level=checkpointing_level)

In [ ]:
test_config = SwinV23DEncoderWithPatchEmbeddingsConfig.model_validate(
    {
        "patch_size": (1, 8, 8),
        "in_channels": 1,
        "use_absolute_position_embeddings": True,
        "learnable_absolute_position_embeddings": False,
        "drop_prob": 0.2,
        "stages": [
            {
                "dim": 36,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
                "attn_drop_prob": 0.2,
                "proj_drop_prob": 0.2,
                "mlp_drop_prob": 0.2,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "in_dim": 36,
                    "out_dim": 72,
                },
                "dim": 72,
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
                "attn_drop_prob": 0.2,
                "proj_drop_prob": 0.2,
                "mlp_drop_prob": 0.2,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "in_dim": 72,
                    "out_dim": 144,
                },
                "dim": 144,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
                "attn_drop_prob": 0.2,
                "proj_drop_prob": 0.2,
            },
        ],
    }
)

test = SwinV23DEncoderWithPatchEmbeddings(test_config)
display(test)
o = test(
    torch.randn(2, 1, 32, 512, 512),
    torch.randn(2, 3),
    crop_offsets=torch.Tensor((10, 10, 10)),
    return_intermediates=True,
)
display((o[0].shape, [x.shape for x in o[1]], [x.shape for x in o[2]]))


SwinV23DEncoderWithPatchEmbeddings(
  (patchify): PatchEmbeddings3D(
    (conv): Conv3d(1, 36, kernel_size=(1, 8, 8), stride=(1, 1, 1), padding=same, bias=False)
    (norm): BatchNorm3d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (checkpointing_level1): ActivationCheckpointing(enabled=False)
  )
  (absolute_position_embeddings): AbsolutePositionEmbeddings3D()
  (encoder): SwinV23DEncoder(
    (stages): ModuleList(
      (0): SwinV23DStage(
        (blocks): ModuleList(
          (0): SwinV23DBlock(
            (w_layer): SwinV23DLayer(
              (transformer): Attention3DWithMLP(
                (attn): Attention3D(
                  (W_q): Linear(in_features=36, out_features=36, bias=True)
                  (W_k): Linear(in_features=36, out_features=36, bias=True)
                  (W_v): Linear(in_features=36, out_features=36, bias=True)
                  (proj): Linear(in_features=36, out_features=36, bias=True)
                  (p

/home/users/arjun.agarwal/miniforge3/envs/env/lib/python3.11/site-packages/torch/nn/modules/conv.py:720: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /pytorch/aten/src/ATen/native/Convolution.cpp:1036.)
  return F.conv3d(



(
    torch.Size([2, 144, 8, 128, 128]),
    [
        torch.Size([2, 32, 512, 512, 36]),
        torch.Size([2, 16, 256, 256, 72]),
        torch.Size([2, 8, 128, 128, 144])
    ],
    [
        torch.Size([2, 32, 512, 512, 36]),
        torch.Size([2, 32, 512, 512, 36]),
        torch.Size([2, 16, 256, 256, 72]),
        torch.Size([2, 16, 256, 256, 72]),
        torch.Size([2, 16, 256, 256, 72]),
        torch.Size([2, 16, 256, 256, 72]),
        torch.Size([2, 16, 256, 256, 72]),
        torch.Size([2, 16, 256, 256, 72]),
        torch.Size([2, 8, 128, 128, 144]),
        torch.Size([2, 8, 128, 128, 144])
    ]
)

# nbdev

In [ ]:
!nbdev_export

# Rough work